Импорт библиотек

In [43]:
import pandas as pd
import requests
import time
import json
from pathlib import Path

#Выгрузка

In [44]:
# Путь к файлу
filepath  = Path('/content/drive/MyDrive/df_dota2_matches.csv')

In [45]:
# Загрузка файла
df = pd.read_csv(filepath)

In [46]:
# Проверка количества строк
df.shape

(1444, 14)

In [47]:
df.tail(10)

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,radiant_team,dire_team
1434,7179531211,6034196670,True,1685530991,811,4464.0,6.0,7,22,81,6,273,"65,2,59,21,68","97,11,44,111,123"
1435,7179528804,6034206609,True,1685530871,1649,4328.0,2.0,7,22,81,7,184,"14,89,35,111,85","71,41,11,40,105"
1436,7179528114,6034201778,True,1685530831,1394,3670.0,3.0,7,22,81,5,191,"46,71,73,86,137","57,68,45,82,32"
1437,7179524213,6034204765,False,1685530620,1649,5034.0,2.0,7,22,81,8,151,"82,85,98,94,91","20,71,119,50,138"
1438,7179521003,6034198498,True,1685530454,1550,4031.0,4.0,7,22,75,8,151,"97,106,108,9,65","40,21,98,123,30"
1439,7180209814,6034799985,True,1685558978,1101,4318.0,4.0,7,22,81,6,184,"83,57,46,87,40","121,98,30,41,64"
1440,7180200307,6034794558,False,1685558497,1157,6446.0,2.0,7,22,81,4,191,"39,4,79,101,71","74,65,97,112,21"
1441,7180199809,6034793018,True,1685558473,1198,2477.0,2.0,7,22,81,6,184,"76,136,101,71,91","86,94,68,105,112"
1442,7180198608,6034792199,False,1685558427,1241,4675.0,3.0,7,22,75,5,271,"42,31,25,22,1","80,5,101,2,8"
1443,7180196219,6034803611,True,1685558312,1967,4505.0,2.0,7,22,81,5,274,"85,86,114,98,39","87,112,104,74,8"


In [48]:
# API запрос с переводом json'а в датафрейм
j = requests.get(f'https://api.opendota.com/api/publicMatches')
publicMatches = j.json()
df_publicmatches = pd.DataFrame.from_dict(publicMatches)

In [49]:
# Фильтрация высокоранговых матчей. Проверка количетсва строк
df_publicmatches_hight_mmr = df_publicmatches.query('avg_rank_tier >= 75')
df_publicmatches_hight_mmr.shape

(5, 14)

In [50]:
# Присоединения новых строк к датафрейму. Проверка количества строк
frames = [df, df_publicmatches_hight_mmr]
df = pd.concat(frames)
df.shape

(1449, 14)

In [51]:
# Так как API запрос парсит недавние 100 матчей, рандомно. То могут встречаться дубликаты. Избавимся от них
df = df.drop_duplicates(subset=['match_id'])

In [52]:
# Сохранение 
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath, index=False)

# Загрузка

In [53]:
# Подключение к БД
# from sqlalchemy import create_engine
# con = create_engine()

In [54]:
# Ускорение выгрузки
# import csv
# from io import StringIO

# def psql_insert_copy(table, conn, keys, data_iter):
#     # gets a DBAPI connection that can provide a cursor
#     dbapi_conn = conn.connection
#     with dbapi_conn.cursor() as cur:
#         s_buf = StringIO()
#         writer = csv.writer(s_buf)
#         writer.writerows(data_iter)
#         s_buf.seek(0)

#         columns = ', '.join('"{}"'.format(k) for k in keys)
#         if table.schema:
#             table_name = '{}.{}'.format(table.schema, table.name)
#         else:
#             table_name = table.name

#         sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
#             table_name, columns)
#         cur.copy_expert(sql=sql, file=s_buf)

In [55]:
# df.to_sql('matches_stat', con, index=False, if_exists='replace', method=psql_insert_copy)

In [56]:
df.sort_values(by='duration', ascending=False)

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,radiant_team,dire_team
560,7167188214,6023138001,True,1684823048,2314,NaN,NaN,7,22,81,4,152,"105,65,4,3,56","97,41,101,40,126"
18,7124385111,5985134051,False,1682404541,2158,5922.0,3.0,7,22,81,7,272,"22,69,88,28,9","104,40,120,110,32"
631,7168625012,6024417183,True,1684910645,2151,4867.0,1.0,7,22,81,9,192,"91,74,4,104,87","14,12,39,7,50"
730,7170270806,6025894677,True,1685011424,2127,4380.0,3.0,7,22,81,6,154,"65,12,75,36,86","79,102,93,126,71"
201,7139638415,5998623274,False,1683273702,2100,5452.0,2.0,7,22,81,6,152,"44,88,86,68,126","11,2,31,56,106"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818,7171518719,6026991798,True,1685084828,503,4883.0,5.0,7,22,81,7,153,"65,94,111,126,104","46,22,78,61,5"
310,7162891415,6019264223,True,1684585612,499,3399.0,3.0,7,22,81,6,274,"64,104,105,71,81","32,61,85,86,97"
420,7164436208,6020652286,False,1684664689,494,6115.0,3.0,7,22,81,6,154,"21,75,76,53,39","65,71,86,40,89"
75,7132541415,5992282197,False,1682861735,417,4806.0,2.0,7,22,81,5,272,"66,74,10,73,137","135,45,120,70,5"
